# 1. Importing the libraries

In [1]:
import pandas as pd
import numpy as np

import warnings
warnings.filterwarnings('ignore')

pd.set_option('display.max_columns', None)
pd.set_option('display.expand_frame_repr', False)
pd.set_option('max_colwidth', -1)

from IPython.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

# 2. Loading the data

In [2]:
df = pd.read_csv('C:/Users/neera/Mad About Sports/Advanced Cricket Analytics Masterclass/Datasets/T20I_ball_by_ball_updated.csv')

# 3. Feature engineering

In [3]:
df.head(2)

,match_id,season,start_date,venue,innings,ball,batting_team,bowling_team,striker,non_striker,bowler,runs_off_bat,extras,wides,noballs,byes,legbyes,penalty,wicket_type,player_dismissed,other_wicket_type,other_player_dismissed
0,211048,2004/05,2005-02-17,Eden Park,1,0.1,Australia,New Zealand,AC Gilchrist,MJ Clarke,DR Tuffey,0,1,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,211048,2004/05,2005-02-17,Eden Park,1,0.2,Australia,New Zealand,AC Gilchrist,MJ Clarke,DR Tuffey,0,1,NaN,NaN,NaN,1.0,NaN,NaN,NaN,NaN,NaN


In [4]:
df.shape

(540332, 22)

In [5]:
# No. of matches
len(df['match_id'].unique())

2365

In [6]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 540332 entries, 0 to 540331
Data columns (total 22 columns):
 #   Column                  Non-Null Count   Dtype  
---  ------                  --------------   -----  
 0   match_id                540332 non-null  int64  
 1   season                  540332 non-null  object 
 2   start_date              540332 non-null  object 
 3   venue                   540332 non-null  object 
 4   innings                 540332 non-null  int64  
 5   ball                    540332 non-null  float64
 6   batting_team            540332 non-null  object 
 7   bowling_team            540332 non-null  object 
 8   striker                 540332 non-null  object 
 9   non_striker             540332 non-null  object 
 10  bowler                  540332 non-null  object 
 11  runs_off_bat            540332 non-null  int64  
 12  extras                  540332 non-null  int64  
 13  wides                   20281 non-null   float64
 14  noballs             

## 3.1 Subsetting only consistent teams

In [7]:
df['batting_team'].unique()

array(['Australia', 'New Zealand', 'England', 'South Africa',
       'West Indies', 'Sri Lanka', 'Pakistan', 'India', 'Kenya',
       'Bangladesh', 'Scotland', 'Zimbabwe', 'Bermuda', 'Ireland',
       'Netherlands', 'Afghanistan', 'Canada', 'Nepal', 'Hong Kong',
       'United Arab Emirates', 'Papua New Guinea', 'Oman', 'ICC World XI',
       'Malaysia', 'Thailand', 'Uganda', 'Botswana', 'Lesotho', 'Malawi',
       'Namibia', 'Sierra Leone', 'Mozambique', 'China', 'Kuwait',
       'United States of America', 'Philippines', 'Vanuatu', 'Nigeria',
       'Tanzania', 'Japan', 'Indonesia', 'Fiji', 'Rwanda', 'Samoa',
       'Ghana', 'Germany', 'Italy', 'Guernsey', 'Jersey', 'Denmark',
       'Norway', 'Mali', 'Maldives', 'Singapore', 'Qatar',
       'Cayman Islands', 'South Korea', 'Portugal', 'Spain', 'Gibraltar',
       'Bhutan', 'Saudi Arabia', 'Bahrain', 'Iran', 'Austria',
       'Isle of Man', 'Belgium', 'Luxembourg', 'Czech Republic',
       'Bulgaria', 'Romania', 'Greece', 'Serbia', '

In [8]:
consistent_teams = [
    'Australia',
    'New Zealand',
    'England',
    'South Africa',
    'West Indies',
    'Sri Lanka',
    'Pakistan',
    'India',
    'Bangladesh'
]

In [9]:
df = df[(df['batting_team'].isin(consistent_teams)) & (df['bowling_team'].isin(consistent_teams))]

In [10]:
df['batting_team'].unique()

array(['Australia', 'New Zealand', 'England', 'South Africa',
       'West Indies', 'Sri Lanka', 'Pakistan', 'India', 'Bangladesh'],
      dtype=object)

In [11]:
df['bowling_team'].unique()

array(['New Zealand', 'Australia', 'England', 'South Africa',
       'West Indies', 'Sri Lanka', 'Pakistan', 'India', 'Bangladesh'],
      dtype=object)

## 3.2 Subsetting the data with required features
### 3.2.1 Sorting by 'match_id' and 'innings'

In [12]:
df = df.sort_values(['match_id', 'innings'], ascending = [True, True])

### 3.2.2 Extracting over and ball no. from 'ball'
#### 3.2.2.1 Renaming 'ball' to 'overs' and 'match_id' to 'id'

In [13]:
df.rename(columns = {
    'match_id': 'id',
    'ball':     'overs'
}, inplace = True)

#### 3.2.2.2 Converting values in 'overs' to string

In [14]:
type(df['overs'][0])

numpy.float64

In [15]:
df['overs'] = df['overs'].astype(str)
type(df['overs'][0])

str

#### 3.2.2.3 Performing operations on one value for testing

In [16]:
# overs, over, ball
# 0.1,   0,    1
df['overs'][0]

'0.1'

In [17]:
df['overs'][0].split('.')

['0', '1']

In [18]:
df['overs'][0].split('.')[0], df['overs'][0].split('.')[1]

('0', '1')

#### 3.2.2.4 Getting over and ball from 'overs'

In [19]:
df['over'] = df['overs'].apply(lambda x: x.split('.')[0])
df['ball'] = df['overs'].apply(lambda x: x.split('.')[1])

### 3.2.3 Extracting total runs from 'runs_off_bat' and 'extras'

In [20]:
df['total_runs'] = df['runs_off_bat'] + df['extras']

### 3.2.4 Getting the required columns

In [21]:
df = df[['id', 'innings', 'batting_team', 'bowling_team', 'overs', 'over', 'ball', 'total_runs', 'player_dismissed']]

### 3.2.5 Replacing NaNs with 0s

In [22]:
df.replace(np.nan, 0, inplace=True)

### 3.2.6 Total runs scored in the innings

In [23]:
df['total'] = df.groupby(['id', 'innings'])['total_runs'].transform('sum')

### 3.2.7 Runs scored till the current ball

In [24]:
df['total_score'] = df.groupby(['id', 'innings'])['total_runs'].apply(lambda x: x.cumsum())

### 3.2.8 Runs scored in previous 30 balls

In [25]:
tmp = df.groupby(['id', 'innings'])['total_runs'].rolling(min_periods = 1, window = 30).sum().reset_index()
tmp

,id,innings,level_2,total_runs
0,211028,1,245,0.0
1,211028,1,246,1.0
2,211028,1,247,1.0
3,211028,1,248,1.0
4,211028,1,249,1.0
...,...,...,...,...
239112,1335808,2,536847,34.0
239113,1335808,2,536848,29.0
239114,1335808,2,536849,29.0
239115,1335808,2,536850,33.0


In [26]:
df['prev_30_runs'] = tmp['total_runs'].to_list()

### 3.2.9 Wickets fallen in previous 30 balls

In [27]:
df['player_dismissed'].unique()

array([0, 'GO Jones', 'A Flintoff', ..., 'CS Fraser', 'Nahida Akter',
       'RS Gayakwad'], dtype=object)

In [28]:
# If player_dismissed == 0, then 0 else 1
df['player_dismissed'] = np.where(df['player_dismissed'] == 0, 0, 1)

In [29]:
df['player_dismissed'].unique()

array([0, 1])

In [30]:
tmp = df.groupby(['id', 'innings'])['player_dismissed'].rolling(min_periods = 1, window = 30).sum().reset_index()

In [31]:
df['prev_30_wickets'] = tmp['player_dismissed'].to_list()

### 3.2.10 Wickets fallen till the current ball

In [32]:
df['total_wickets'] = df.groupby(['id', 'innings'])['player_dismissed'].apply(lambda x: x.cumsum())

### 3.2.11 Dot balls in previous 30 balls

In [33]:
df['prev_30_dot_balls'] = df['total_runs']
df['prev_30_dot_balls'] = np.where(df['prev_30_dot_balls'] == 0, 1, 0)

tmp = df.groupby(['id', 'innings'])['prev_30_dot_balls'].rolling(min_periods = 1, window = 30).sum().reset_index()
df['prev_30_dot_balls'] = tmp['prev_30_dot_balls'].to_list()

## 3.2.12 Boundaries in previous 30 balls

In [34]:
df['prev_30_boundaries'] = df['total_runs']
df['prev_30_boundaries'] = np.where(df['prev_30_boundaries'] > 3, 1, 0)

tmp = df.groupby(['id', 'innings'])['prev_30_boundaries'].rolling(min_periods = 1, window = 30).sum().reset_index()
df['prev_30_boundaries'] = tmp['prev_30_boundaries'].to_list()

### 3.2.13 Converting 'prev_30' fields to int

In [35]:
convert_dict = {
    'prev_30_runs':       int,
    'prev_30_wickets':    int,
    'prev_30_dot_balls':  int,
    'prev_30_boundaries': int
}

df = df.astype(convert_dict)

### 3.2.14 Reordering the columns

In [36]:
df.columns

Index(['id', 'innings', 'batting_team', 'bowling_team', 'overs', 'over',
       'ball', 'total_runs', 'player_dismissed', 'total', 'total_score',
       'prev_30_runs', 'prev_30_wickets', 'total_wickets', 'prev_30_dot_balls',
       'prev_30_boundaries'],
      dtype='object')

In [37]:
columns = ['id', 'innings', 'batting_team', 'bowling_team', 'overs','over', 'ball', 'total_runs', 'player_dismissed', 'total', 'total_score', 'total_wickets',
'prev_30_runs', 'prev_30_wickets', 'prev_30_dot_balls', 'prev_30_boundaries']

df = df[columns]

df.head(40)

,id,innings,batting_team,bowling_team,overs,over,ball,total_runs,player_dismissed,total,total_score,total_wickets,prev_30_runs,prev_30_wickets,prev_30_dot_balls,prev_30_boundaries
245,211028,1,England,Australia,0.1,0,1,0,0,179,0,0,0,0,1,0
246,211028,1,England,Australia,0.2,0,2,1,0,179,1,0,1,0,1,0
247,211028,1,England,Australia,0.3,0,3,0,0,179,1,0,1,0,2,0
248,211028,1,England,Australia,0.4,0,4,0,0,179,1,0,1,0,3,0
249,211028,1,England,Australia,0.5,0,5,0,0,179,1,0,1,0,4,0
250,211028,1,England,Australia,0.6,0,6,1,0,179,2,0,2,0,4,0
251,211028,1,England,Australia,0.7,0,7,2,0,179,4,0,4,0,4,0
252,211028,1,England,Australia,1.1,1,1,0,0,179,4,0,4,0,5,0
253,211028,1,England,Australia,1.2,1,2,0,0,179,4,0,4,0,6,0
254,211028,1,England,Australia,1.3,1,3,1,0,179,5,0,5,0,6,0


# 4. Exporting the df

In [38]:
df.to_csv('C:/Users/neera/Mad About Sports/Advanced Cricket Analytics Masterclass/Datasets/T20I_data_with_features.csv', index = None)